# Timescale Vector Store (PostgreSQL)

This notebook shows how to use the Postgres vector store `TimescaleVector` to store and query vector embeddings.

## What is Timescale Vector?
**[Timescale Vector](https://www.timescale.com/ai) is PostgreSQL++ for AI applications.**

Timescale Vector enables you to efficiently store and query billions of vector embeddings in `PostgreSQL`.
- Enhances `pgvector` with faster and more accurate similarity search on 1B+ vectors via DiskANN inspired indexing algorithm.
- Enables fast time-based vector search via automatic time-based partitioning and indexing.
- Provides a familiar SQL interface for querying vector embeddings and relational data.

Timescale Vector scales with you from POC to production:
- Simplifies operations by enabling you to store relational metadata, vector embeddings, and time-series data in a single database.
- Benefits from rock-solid PostgreSQL foundation with enterprise-grade feature liked streaming backups and replication, high-availability and row-level security.
- Enables a worry-free experience with enterprise-grade security and compliance.

## How to use Timescale Vector
Timescale Vector is available on [Timescale](https://www.timescale.com/products), the cloud PostgreSQL platform. (There is no self-hosted version at this time.)

- LlamaIndex users get a 90-day free trial for Timescale Vector.
- To get started, [signup](https://console.cloud.timescale.com/signup) to Timescale, create a new database and follow this notebook!
- See the [installation instructions](https://github.com/timescale/python-vector) for more details on using Timescale Vector in python.

## Setup
Let's import everything we'll need for this notebook.

In [17]:
# import logging
# import sys

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import SimpleDirectoryReader, StorageContext
from llama_index.indices.vector_store import VectorStoreIndex
from llama_index.vector_stores import TimescaleVectorStore
import textwrap
import openai

### Setup OpenAI API Key
To create embeddings for documents loaded into the index, let's configure your OpenAI API key:

In [18]:
# Get openAI api key by reading local .env file
# The .env file should contain a line starting with `OPENAI_API_KEY=sk-`
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# OR set it explicitly 
#import os
#os.environ["OPENAI_API_KEY"] = "<your key>"

openai.api_key = os.environ["OPENAI_API_KEY"]

### Loading documents
For this example, we'll use a [SimpleDirectoryReader](https://gpt-index.readthedocs.io/en/stable/examples/data_connectors/simple_directory_reader.html) to load the documents stored in the the `paul_graham_essay` directory. 

The `SimpleDirectoryReader` is one of LlamaIndex's most commonly used data connectors to read one or multiple files from a directory.

In [19]:
# load sample data from the data directory using a SimpleDirectoryReader
documents = SimpleDirectoryReader("../data/paul_graham").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: ecd1868c-7484-4aab-b7ba-6c70e0ae27df


### Create a PostgreSQL database and get a Timescale service URL
You need a service url to connect to your Timescale database instance.

To connect to your cloud PostgreSQL database in Timescale, you'll need your service URI, which can be found in the cheatsheet file you downloaded after creating a new database. The URI will look something like this: `postgres://tsdbadmin:<password>@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require`

In [20]:
# Get the service url by reading local .env file
# The .env file should contain a line starting with `TIMESCALE_SERVICE_URL=postgresql://`
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

TIMESCALE_SERVICE_URL = os.environ["TIMESCALE_SERVICE_URL"]

# OR set it explicitly
# TIMESCALE_SERVICE_URL = "postgres://tsdbadmin:<password>@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require"

### Create a VectorStore Index with the TimescaleVectorStore
Next, to perform a similarity search, we first create a `TimescaleVector` [vector store](https://gpt-index.readthedocs.io/en/stable/core_modules/data_modules/storage/vector_stores.html) to store our vector embeddings from the essay content. TimescaleVectorStore takes a few arguments, namely the `service_url` which we loaded above, along with a `table_name` which we will be the name of the table that the vectors are stored in.

Then we create a [Vector Store Index](https://gpt-index.readthedocs.io/en/stable/community/integrations/vector_stores.html#vector-store-index) on the documents backed by Timescale using the previously documents.

In [ ]:
# Create a TimescaleVectorStore to store the documents
vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="paul_graham_essay",
)

# Create a new VectorStoreIndex using the TimescaleVectorStore
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

### Query the index
Now that we've indexed the documents in our VectorStore, we can ask questions about our documents in the index by using the default `query_engine`.

Note you can also configure the query engine to configure the top_k most similar results returned, as well as metadata filters to filter the results by. See the [configure standard query setting section](https://gpt-index.readthedocs.io/en/stable/core_modules/data_modules/index/vector_store_guide.html) for more details.

In [22]:
query_engine = index.as_query_engine()
response = query_engine.query("Did the author work at YC?")

In [23]:
print(textwrap.fill(str(response), 100))

Yes, the author did work at YC.


In [24]:
response = query_engine.query("What did the author do growing up?")

In [25]:
print(textwrap.fill(str(response), 100))

The author worked on writing and programming growing up. They wrote short stories and tried writing
programs on an IBM 1401 computer. They also built a microcomputer and started programming on it,
writing simple games and a word processor.


### Querying existing index
In the example above, we created a new vectorstore and index from documents we loaded. Next we'll look at how to query an existing index.

In [ ]:
vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="paul_graham_essay",
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")

In [27]:
print(textwrap.fill(str(response), 100))

The author worked on writing and programming outside of school. They wrote short stories and tried
writing programs on an IBM 1401 computer. They also built a microcomputer and started programming on
it, writing simple games and a word processor.
